In [22]:
import errno

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')
import spacy
from spacy.language import Language
from spacy import displacy
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher
from spacy.matcher import Matcher
from spacy_wordnet.wordnet_annotator import WordnetAnnotator
from nltk.tokenize import word_tokenize
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/bal

In [23]:
@Language.factory('make_street_entity_matcher')
def make_street_entity_matcher(nlp, name, entity_name:str, file_name:str, pattern=None):
    return EntityMatcher(nlp, file_name, entity_name, pattern)

ValueError: [E004] Can't set up pipeline component: a factory for 'make_street_entity_matcher' already exists. Existing factory: <function make_street_entity_matcher at 0x13114c378>. New factory: <function make_street_entity_matcher at 0x1395702f0>

In [ ]:
@Language.factory('make_city_entity_matcher')
def make_city_entity_matcher(nlp, name,entity_name:str, file_name=None, pattern=None):
    return EntityMatcher(nlp, file_name, entity_name, pattern)

In [ ]:
@Language.factory('make_landmark_entity_matcher')
def make_directions_entity_matcher(nlp, name,entity_name:str, file_name=None, pattern=None):
    return EntityMatcher(nlp, file_name, entity_name, pattern)

In [ ]:
@Language.factory('make_directions_entity_matcher')
def make_directions_entity_matcher(nlp, name,entity_name:str, file_name=None, pattern=None):
    return EntityMatcher(nlp, file_name, entity_name, pattern)

In [ ]:
@Language.factory('make_neighbourhood_entity_matcher')
def make_neighbourhood_entity_matcher(nlp, name,entity_name:str, file_name=None, pattern=None):
    return EntityMatcher(nlp, file_name, entity_name, pattern)

In [ ]:
@Language.factory('make_ordinals_entity_matcher')
def make_ordinals_entity_matcher(nlp, name,entity_name:str, file_name=None, pattern=None):
    return EntityMatcher(nlp, file_name, entity_name, pattern)

In [ ]:
@Language.factory('make_organisation_entity_matcher')
def make_organisation_entity_matcher(nlp, name,entity_name:str, file_name=None, pattern=None):
    return EntityMatcher(nlp, file_name, entity_name, pattern)

In [ ]:
class EntityMatcher:
    def __init__(self, nlp, file_name, entity_name, pattern):
        if file_name:
            try:
                line_list = open(file_name,'r').readlines()

            except IOError as ex:
                if ex.errno != errno.ENOENT:
                    raise
            else:
                phrase_list = []
                for l in line_list:
                    phrase_list.append(l.strip())
                phrase_patterns = [nlp(text) for text in phrase_list]
                self.phrase_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
                print(phrase_patterns)
                self.phrase_matcher.add(entity_name, None, *phrase_patterns )
        if pattern:
            self.matcher = Matcher(nlp.vocab)
            self.matcher.add(entity_name, pattern )

    def __call__(self, doc):
        matches = set()
        if hasattr(self, "matcher"):
            matches.update(self.matcher(doc))
        if hasattr(self, "phrase_matcher"):
            matches.update(self.phrase_matcher(doc))
        seen_tokens = set()
        
        new_entities = []
        entities = doc.ents
        for match_id, start, end in matches:
            if start not in seen_tokens and end - 1 not in seen_tokens:
                new_entities.append(Span(doc, start, end, label=match_id))
                entities = [
                    e for e in entities if not (e.start < end and e.end > start)
                ]
                seen_tokens.update(range(start, end))

        doc.ents = tuple(entities) + tuple(new_entities)
        return doc

In [ ]:
def regex_part_of_string(filename):
    try:
        line_list = open(filename,'r').readlines()
    except IOError as ex:
        if ex.errno != errno.ENOENT:
            raise
    else:
        phrase_list = '('
        for l in line_list:
            if phrase_list == '(':
                phrase_list += f'{l.strip()})'
            else:
                phrase_list += f'|({l.strip()})'
        return f"(\w*({phrase_list})\w*)"

In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("spacy_wordnet", after='tagger', config={'lang': nlp.lang})
matcher = Matcher(nlp.vocab)

In [ ]:

filename_dutch_streets_types = '../../Data/street_types_dutch.txt'
street_regex = regex_part_of_string(filename_dutch_streets_types)
regex =[[{"TEXT": {"REGEX": "^[A-Z][0-9]{1,5}.*"}}],[{"TEXT": {"REGEX": street_regex}}]]
filename_street = '../../Data/street_names_north_holland.txt'
nlp.add_pipe("make_street_entity_matcher", config={"file_name":filename_street,"entity_name": "STREET", "pattern": regex}, after="ner")

[A A de Lannoy-Willemsstr, A Eduardine Kokplnts, A. Reitsmakade, A.A.H. Struijckenkade, A.J. Koejemansstraat, A.J. Krophollerstraat, A.M. de Jongstraat, A.S. Onderwijzerhof, A.W. Grootehof, Aaf Bouberstraat, Aagtdorperpad, Aakstraat, Aalbersestraat, Aalsmeerplein, Aalsmeerweg, Aambeeldstraat, Aandewind, Aardbeistraat, Aart van der Leeuwstraat, Abbenesstraat, Abberdaan, Abcouderstraatweg, Abebe Bikilalaan, Abraham de Haanstraat, Abraham Dudok v Heelstr, Abraham Kuyperplein, Abraham Staalmanplein, Abraham van der Hartstr, Abrikozenstraat, Accraweg, Accumulatorweg, Achillesstraat, Achter Oosteinde, Achtergouwtje, Achtergracht, Achtergracht AB, Achterlaan, Achtersteven, Adam van Germezhof, Adamello, Adelaarsweg, Adele Opzoomerstraat, Adingerdorphof, Admiraal de Ruijterweg, Admiraal Helfrichstraat, Admiralengracht, Admiraliteitstraat, Adolf van Nassaustraat, Adriaan Dorsmanstraat, Adriaan Kluitstraat, Adriaan Loosjesstraat, Adriaan van Bergenstraat, Adriaan van der Horststr, Adriaan van Oor

In [ ]:
filename_city = '../../Data/cities.txt'
nlp.add_pipe("make_city_entity_matcher", config={ "file_name": filename_city, "entity_name": "CITY",}, after="ner")

[Aadorp, Aagtdorp, Aagtekerke, Aalbeek, Aalburg, Aalden, Aalsmeer, Aalsmeerderbrug, Aalst, Buren, Aalst, North Brabant, Aalst, Zaltbommel, Aalsum, Friesland, Aalsum, Groningen, Aalten, Aardenburg, Aarlanderveen, Aarle-Rixtel, Aartswoud, Abbega, Abbenbroek, Abbenes, Abcoude, Absdale, Abshoven, Achlum, Achterberg, Achterste Erm, Achterveld, Achthuizen, Achtmaal, Achttienhoven, South Holland, Achttienhoven, Utrecht, Acquoy, Adorp, Aduard, Aduarderzijl, Aegum, Aerdenhout, Aerdt, Afferden, Gelderland, Afferden, Limburg, Agelo, Aijen, Akersloot, Akkrum, Akmarijp, Albergen, Alblasserdam, Aldeboarn, Alem, Alkmaar, Allingawier, Almelo, Almen, Almere, Almkerk, Alphen aan den Rijn, Alphen, Gelderland, Alphen, North Brabant, Alteveer, Roden, Alteveer, Zuidwolde, Alteveer, Groningen, Altforst, Alverna, Ambt Delden, Amby, Ameide, Ameland, Amen, America, Amerongen, Amersfoort, Ammerstol, Ammerzoden, Ampsen, Amstelhoek, Amstelveen, Amstenrade, Amsterdam, Amsweer, Andel, Andelst, Anderen, Andijk, Ane, 

In [ ]:
filename_dutch_landmark_types = '../../Data/landmark_types_dutch.txt'
landmark_regex = regex_part_of_string(filename_dutch_landmark_types)
regex =[[{"TEXT": {"REGEX": landmark_regex}}]]
filename_landmarks = '../../Data/landmarks.txt'
nlp.add_pipe("make_landmark_entity_matcher", config={ "file_name": filename_landmarks, "entity_name": "LANDMARK", "pattern": regex }, after="ner")


[traffic light, traffic lights, bridge, viaduct, road, gas station, tunnel, roundabout, hospital, park, flat, house, speedbump]


In [ ]:
filename_landmarks = '../../Data/neighbourhoods_amsterdam.txt'
nlp.add_pipe("make_neighbourhood_entity_matcher", config={ "file_name": filename_landmarks, "entity_name": "NEIGHBOURHOOD",}, after="ner")


[Centre, Centrum, West, Niuew-west, Noord, Oost, Zuid, Zuidoost, Centrum-Oost, Centrum-West, Geuzenveld, Slotermeer, Sloterdijken, Slotervaart, De Aker, Sloten and Nieuw-Sloten, Osdorp, Noord-West, Banne Buiksloot, Kadoelen, Oostzanerwerf, Tuindorp Oostzaan, Noord-Oost, Buikslotermeer, Elzenhagen, Waterland, Waterlandpleinbuurt, Oud-Noord, Buiksloterdijk/Nieuwendammerdijk, IJplein/Vogelbuurt, Noordelijke IJ-oever, Tuindorp Buiksloot, Tuindorp Nieuwendam, Volewijck, Indische Buurt, IJburg and Zeeburgereiland, Oud-Oost, Watergraafsmeer, Oostelijk Havengebied, De Baarsjes, Oud-West, Westerpark, Bos en Lommer, Buitenveldert, Zuidas, Rivierenbuurt, De Pijp, Oud-Zuid, Amstel lll en ArenAPoort, Bijlmer-Centrum, Gaasperdam Driemond, Bijlmer-Oost]


In [ ]:
filename_landmarks = '../../Data/ordinals.txt'
nlp.add_pipe("make_ordinals_entity_matcher", config={ "file_name": filename_landmarks, "entity_name": "ORDINALS",}, after="ner")

[1st, 2nd, 3rd, 4th, 5th, 6th, 7th, 8th, first, second, third, last, next, end, before, after, then, immediately, Immediately, over]


In [ ]:
filename_landmarks = '../../Data/organisations.txt'
nlp.add_pipe("make_organisation_entity_matcher", config={ "file_name": filename_landmarks, "entity_name": "ORGANISATION",}, after="ner")


[AH, Albert Heijn, Albert Heijn branch, Albert Heijn shop, shop]


In [ ]:
directions =[[{"LOWER": 'left'}], [{"LOWER": 'right'}], [{"LOWER": 'straight'}]]
nlp.add_pipe("make_directions_entity_matcher", config={"entity_name": "DIRECTION", "pattern": directions}, after="ner")

In [ ]:
file = '../../Data/AH/Haarlemmerdijk 1.txt'
tokenisedList = []
with open(file,'r') as f:
    line = f.readline()
    is_travel_direction = False
    while line:
        line = f.readline()
        nlpLine = nlp(line)
        pattern = [{"LOWER": "directions"}]
        matcher.add("BEGIN", [pattern])
        matches = matcher(nlpLine)

        if line == '\n':
            is_travel_direction = False
        if is_travel_direction:
            tokenisedList.append(nlpLine)
        for match_id, _, _ in matches:
            string_id = nlp.vocab.strings[match_id]
            if string_id == 'BEGIN':
                is_travel_direction = True

In [ ]:
# if you want to keep specific stop words
# nlp.Defaults.stop_words.remove()
# nlp.vocab['beyond'].is_stop = False

In [ ]:
colors = {"STREET": '#981231',
          "DIRECTION": '#33DCFF',
          "CITY":'#742318',
          "LANDMARK":'#ba03fc',
          "NEIGHBOURHOOD":'#1212cc',
          "ORDINALS":'#cc1212'}
options={'distance':50,'colors': colors}
displacy.render(tokenisedList, style='ent',options=options, jupyter=True)
displacy.render(tokenisedList, style="dep")
#displacy.serve(tokenisedList, style='ent', options=options )

In [ ]:
class RouteDescription:
    def __init__(self, begin, end):
        self.start =

    def __call__(self, doc):

